In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import matplotlib.pyplot as plt

In [2]:
def dataclean(zillow_yvalues_data,census_data,date):
    # takes in the zillow data with all the years and acs census data of two years before 
    # year to predict and date formatted '2020-09' and joins the census on zillow and cleans 
    
    
    zillow_yvalues_data_yearsubsect=zillow_yvalues_data[['RegionName','City','State',date]]
    fulldata=zillow_yvalues_data_yearsubsect.merge(census_data,how='left',left_on='RegionName', right_on='geo_id' ).drop(['geo_id'],axis=1)
    fulldata=fulldata[fulldata[date].isna()!=1]
    #drop column if >20% are nas 
    cols_to_drop=['do_date']
    for i in fulldata.columns:

        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    print(cols_to_drop)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    #drop rows with more than 50% nas
    #in this jan2020was one row
    fulldata=fulldata.loc[fulldata.isna().sum(axis=1)<.5*len(fulldata.columns)]
    #num of missing values in columns
    countofcols=0
    cols_with_na=[]
    for i in fulldata.columns:
        if i in cols_to_drop:
            print(i)
        elif sum(fulldata[i].isna())>0:
#             print('Column:',i)
#             print('Num of NAs:',sum(fulldata[i].isna()))
#             print('Num of 0s:',sum(fulldata[i]==0))
            cols_with_na.append(i)
            countofcols+=1
    #fill in nas with avrg of the column for the city 
    for i in cols_with_na:
        for indexofna in (list(fulldata[i][fulldata[i].isna()].index)):

            if bool(~(fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()==fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean())):
                fulldata.loc[indexofna,i]=fulldata[fulldata.State==fulldata.loc[indexofna,'State']][i].mean()
                
            else:
                fulldata.loc[indexofna,i]=fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()
                
    
    return fulldata

In [3]:
#fulldata=dataclean(zillow_yvalues_data,census_data,date)

# Assembling training data w yvals from 2011-2017 and before 
-means census data from 2015 and before 


In [4]:
def assemble_train_test_predict(years_out=1,year_gap_in_data=2):
    
    df = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental-2.csv')
    zillow_yvalues_data=pd.merge(df.iloc[:,0:7], df.iloc[:,8:].interpolate(axis=1), left_index=True, right_index=True)
    genericfilepath='acs_data/acs20??.csv'
    census_data_years=list(range(12,19,1))
    years=list(range(14,21,1))
    years_for_train=years[:-years_out]
    years_for_pred=years[-years_out:]
    census_data_years_train=census_data_years[:-years_out]
    print(years_for_train)
    print(census_data_years_train)
    olddatedata=pd.DataFrame()
    for i in years_for_train:
        print(i)
        tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data))
        yearnumber=str(i)
        if len(yearnumber)<2:
            yearnumber='0'+yearnumber
        date='20'+yearnumber+'-09'
#         print('date',date)
#         print('tempfilepath',tempfilepath)
        try:
            census_data= pd.read_csv(tempfilepath)
            datedata=dataclean(zillow_yvalues_data,census_data,date)
            datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
            datedata=datedata.rename(columns={date: 'Rent'})
            olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)
        except:
            p=1
    
    
    fulldata=olddatedata
    cols_to_drop=[]
    for i in fulldata.columns:
        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    
    
#     print('\n\npred\n\n')
    
    #assemble prediction dataset
    olddatedata=pd.DataFrame()
    for i in years_for_pred:
#         print(i)
        tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data))
        yearnumber=str(i)
        if len(yearnumber)<2:
            yearnumber='0'+yearnumber
        date='20'+yearnumber+'-09'
        if date=='2020-09':
            date='2020-01'
#         print('date',date)
#         print('tempfilepath',tempfilepath)
        
        census_data= pd.read_csv(tempfilepath)
        datedata=dataclean(zillow_yvalues_data,census_data,date)
        datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
        datedata=datedata.rename(columns={date: 'Rent'})
#         newfilename='Data_acsPREDICT20'+yearnumber+'.csv'
#         datedata.to_csv(newfilename)
        olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)
        
    
    
    preddata=olddatedata
    cols_to_drop=[]
    for i in preddata.columns:
        if sum(preddata[i].isna())/len(preddata)> .2:
            cols_to_drop.append(i)
    preddata=preddata.drop(cols_to_drop,axis=1)
    
    
    #ensures prediciton and training set have same cols 
    cols_in_common=list(fulldata.columns & preddata.columns)
    print(len(preddata.columns))
    print(len(cols_in_common))
    preddata=preddata[cols_in_common]
    fulldata=fulldata[cols_in_common]

    
    
#     #train test split
    tdata_train, tdata_test = train_test_split(fulldata, test_size=0.33, random_state=100)
    tdata_train.to_csv('Data_acsTRAIN.csv',index=False)
    tdata_test.to_csv('Data_acsTEST.csv',index=False)
    preddata.to_csv('Data_acsPREDICT.csv',index=False)
    
    return preddata
    


In [5]:
assemble_train_test_predict(years_out=1,year_gap_in_data=2)


[14, 15, 16, 17, 18, 19]
[12, 13, 14, 15, 16, 17]
14
['do_date']
15
['do_date']
16
['do_date']
17
['do_date', 'geoid']
18
['do_date', 'pop_5_years_over', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced']
19
['do_date', 'pop_5_years_over', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced']
['do_date', 'pop_5_years_over', 'pop_15_and_over', 'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced', 'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english']
232
232


,RegionName,City,State,Rent,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,...,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race
0,10025-2020-01,New York,NY,3676.0,22248.0,19662.0,0.0,1499.0,7255.0,2293.0,...,5748.0,1721.0,15509.0,5086.0,70558.0,6257.0,5283.0,1646.0,42055.0,21736.0
1,60657-2020-01,Chicago,IL,1731.0,25196.0,11428.0,1958.0,893.0,3159.0,1415.0,...,2105.0,1077.0,11793.0,3242.0,53263.0,8384.0,4818.0,1640.0,46468.0,4901.0
2,10023-2020-01,New York,NY,3561.0,20157.0,13536.0,1941.0,853.0,3194.0,1238.0,...,2914.0,915.0,15213.0,2738.0,50726.0,3359.0,2266.0,979.0,31688.0,6562.0
3,77494-2020-01,Katy,TX,1286.0,4825.0,30517.0,2009.0,324.0,1358.0,495.0,...,6213.0,2638.0,13063.0,9765.0,69744.0,5312.0,9007.0,2701.0,47526.0,24895.0
4,60614-2020-01,Chicago,IL,1916.0,21566.0,12018.0,1964.0,626.0,2819.0,1105.0,...,2333.0,776.0,11520.0,2448.0,50765.0,5092.0,4130.0,1529.0,43641.0,4317.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,75202-2020-01,Dallas,TX,1739.0,927.0,417.0,1979.0,5.0,191.0,60.0,...,47.0,43.0,329.0,207.0,1675.0,72.0,45.0,231.0,1386.0,250.0
1854,84627-2020-01,Ephraim,UT,1298.0,548.0,1425.0,1988.0,30.0,115.0,36.0,...,622.0,283.0,234.0,528.0,2961.0,40.0,244.0,1613.0,2853.0,760.0
1855,60301-2020-01,Oak Park,IL,1625.0,1115.0,480.0,1982.0,18.0,351.0,161.0,...,80.0,74.0,470.0,237.0,2197.0,138.0,219.0,32.0,1394.0,309.0
1856,84647-2020-01,Mount Pleasant,UT,1221.0,444.0,1041.0,1979.0,62.0,102.0,6.0,...,848.0,256.0,158.0,397.0,2746.0,83.0,185.0,771.0,1800.0,139.0


## Column Dropping 


In [20]:
traindata=pd.read_csv('Data_acsTRAIN.csv')
testdata=pd.read_csv('Data_acsTEST.csv')
predictdata=pd.read_csv('Data_acsPREDICT.csv')

In [21]:
#list(traindata.columns)


In [22]:
cols_to_drop=[
    
 'nonfamily_households',
 'family_households',
 'median_year_structure_built',
 'rent_burden_not_computed',
 'rent_over_50_percent',
 'rent_40_to_50_percent',
 'rent_35_to_40_percent',
 'rent_30_to_35_percent',
 'rent_25_to_30_percent',
 'rent_20_to_25_percent',
 'rent_15_to_20_percent',
 'rent_10_to_15_percent',
 'rent_under_10_percent',

 'male_pop',
 'female_pop',
 'white_pop',
 'amerindian_pop',
 'other_race_pop',
 'two_or_more_races_pop',
 'not_hispanic_pop',

 'households',
 'vacant_housing_units',
 'median_rent',
 'percent_income_spent_on_rent',
 'owner_occupied_housing_units',

 'families_with_young_children',
 'two_parent_families_with_young_children',
 'two_parents_in_labor_force_families_with_young_children',
 'two_parents_father_in_labor_force_families_with_young_children',
 'two_parents_mother_in_labor_force_families_with_young_children',
 'two_parents_not_in_labor_force_families_with_young_children',
 'one_parent_families_with_young_children',
 'father_one_parent_families_with_young_children',
 'father_in_labor_force_one_parent_families_with_young_children',
 'commute_10_14_mins',
 'commute_15_19_mins',
 'commute_20_24_mins',
 'commute_25_29_mins',
 'commute_30_34_mins',
 'commute_45_59_mins',
 'aggregate_travel_time_to_work', 
 'renter_occupied_housing_units_paying_cash_median_gross_rent',

 'occupied_housing_units',

 'dwellings_1_units_detached',
 'dwellings_1_units_attached',
 'dwellings_2_units',
 'dwellings_3_to_4_units',
 'dwellings_5_to_9_units',
 'dwellings_10_to_19_units',
 'dwellings_20_to_49_units',
 

 'housing_built_2000_to_2004', 
 
 'male_under_5',
 'female_under_5',
 'employed_pop',
 'unemployed_pop', 
 'not_in_labor_force',
 'pop_16_over',
 'pop_in_labor_force',
 'asian_male_45_54',
 'asian_male_55_64',
 'black_male_45_54',
 'black_male_55_64',
 'hispanic_male_45_54',
 'hispanic_male_55_64',
 'white_male_45_54',
 'white_male_55_64', 
 'bachelors_degree_2',
 'bachelors_degree_or_higher_25_64',
 'children',
 'commuters_by_bus',
 'commuters_by_car_truck_van',
 'commuters_by_carpool',
 'commuters_by_subway_or_elevated',
 'commuters_drove_alone',
 'four_more_cars',
 'group_quarters',
 'households_public_asst_or_food_stamps',
 'in_grades_1_to_4',
 'in_grades_5_to_8',
 'in_grades_9_to_12',
 'less_than_high_school_graduate',
 'male_45_64_associates_degree',
 'male_45_64_bachelors_degree',
 'male_45_64_graduate_degree',
 'male_45_64_less_than_9_grade',
 'male_45_64_grade_9_12',
 'male_45_64_high_school',
 'male_45_64_some_college',
 'male_45_to_64', 
 'management_business_sci_arts_employed',
 'graduate_professional_degree', 
 'in_school',
 'no_car',
 'no_cars',  
 'one_car',
 'two_cars',
 'three_cars',
 
 'population_1_year_and_over',
 'population_3_years_over', 
 'workers_16_and_over',
 'high_school_diploma',
 'less_one_year_college',
 'one_year_more_college',
    
    
 'pop_25_years_over',
 'commute_35_44_mins',
 'commuters_16_over',
    
    
 'pop_determined_poverty_status',
 'hispanic_any_race',
    'occupation_natural_resources_construction_maintenance',
 'occupation_production_transportation_material',
 'occupation_services',
'male_male_households',
'female_female_households',
      
    ##now dropping in response to vif/multicolineariy :
    #can try w or without dropping these 
    
    
'civilian_labor_force',
    'sales_office_employed',
    
  

'employed_agriculture_forestry_fishing_hunting_mining',
 'employed_arts_entertainment_recreation_accommodation_food',
 'employed_construction',
 'employed_education_health_social',
 'employed_finance_insurance_real_estate',
 'employed_information',
 'employed_manufacturing',
 'employed_other_services_not_public_admin',
 'employed_public_administration',
 'employed_retail_trade',
 'employed_science_management_admin_waste',
 'employed_transportation_warehousing_utilities',
 'employed_wholesale_trade',
'male_under_5',
 'male_5_to_9',
 'male_10_to_14',
 'male_15_to_17',
 'male_18_to_19',
 'male_20',
 'male_21',
 'male_22_to_24',
 'male_25_to_29',
 'male_30_to_34',
 'male_35_to_39',
 'male_40_to_44',
 'male_45_to_49',
 'male_50_to_54',
 'male_55_to_59',
 'male_65_to_66',
 'male_67_to_69',
 'male_70_to_74',
 'male_75_to_79',
 'male_80_to_84',
 'male_85_and_over',
 'female_under_5',
 'female_5_to_9',
 'female_10_to_14',
 'female_15_to_17',
 'female_18_to_19',
 'female_20',
 'female_21',
 'female_22_to_24',
 'female_25_to_29',
 'female_30_to_34',
 'female_35_to_39',
 'female_40_to_44',
 'female_45_to_49',
 'female_50_to_54',
 'female_55_to_59',
 'female_60_to_61',
 'female_62_to_64',
 'female_65_to_66',
 'female_67_to_69',
 'female_70_to_74',
 'female_75_to_79',
 'female_80_to_84',
 'female_85_and_over',
'housing_units_renter_occupied','mortgaged_housing_units',
    'owner_occupied_housing_units_median_value',
'housing_units','occupation_management_arts','married_households',
    'some_college_and_associates_degree',
   
    'occupation_sales_office',
    'income_less_10000',
 'income_10000_14999',
 'income_15000_19999',
 'income_20000_24999',
 'income_25000_29999',
 'income_30000_34999',
 'income_35000_39999',
 'income_40000_44999',
 'income_45000_49999',
 'income_50000_59999',
 'income_60000_74999',
 'income_75000_99999',
 'income_100000_124999',
 'income_125000_149999',
 'income_150000_199999',
 'income_200000_or_more',
   
    'in_undergrad_college',
    'associates_degree',
    'owner_occupied_housing_units_lower_value_quartile',
    'owner_occupied_housing_units_upper_value_quartile',
    'children_in_single_female_hh','not_us_citizen_pop',
    'black_pop', 'asian_pop', 'hispanic_pop',
       'commuters_by_public_transportation',
'vacant_housing_units_for_rent',
       'vacant_housing_units_for_sale',
    'bachelors_degree', 'masters_degree',
       'commute_60_more_mins', 'commute_less_10_mins','worked_at_home','income_per_capita',
    'pop_25_64',
    'high_school_including_ged','different_house_year_ago_same_city','different_house_year_ago_different_city','housing_built_2005_or_later'

    
]

In [23]:

def combo_and_drop_acs(Tempdata,cols_to_drop):
    Tempdata['Percent_Female']=Tempdata.female_pop/Tempdata.total_pop
    Tempdata['families_with_young_children_summed']=Tempdata['two_parent_families_with_young_children']+Tempdata['two_parents_in_labor_force_families_with_young_children']+Tempdata['two_parents_father_in_labor_force_families_with_young_children']+Tempdata['two_parents_mother_in_labor_force_families_with_young_children']+Tempdata['two_parents_not_in_labor_force_families_with_young_children']+Tempdata['one_parent_families_with_young_children']+Tempdata['father_one_parent_families_with_young_children']+Tempdata['father_in_labor_force_one_parent_families_with_young_children']+Tempdata['families_with_young_children']
    Tempdata['dwellings_1_units']=Tempdata['dwellings_1_units_detached']+Tempdata['dwellings_1_units_attached']
    Tempdata['dwellings_under_10_units']=Tempdata['dwellings_2_units']+Tempdata['dwellings_5_to_9_units']+Tempdata['dwellings_3_to_4_units']
    Tempdata['unemployment_rate']=Tempdata['unemployed_pop']/Tempdata['civilian_labor_force']
    Tempdata['average_car_per_household']=(Tempdata['four_more_cars']*4+Tempdata['three_cars']*3+Tempdata['two_cars']*2+Tempdata['one_car'])/Tempdata.households
    Tempdata['Perc_commuters_under_15_min']=(Tempdata.commute_10_14_mins+Tempdata.commute_less_10_mins)/Tempdata.commuters_16_over
    Tempdata['Perc_commuters_over_45_min']=(Tempdata.commute_60_more_mins+Tempdata.commute_45_59_mins)/Tempdata.commuters_16_over
    Tempdata['Percent_pop_25_64']=Tempdata.pop_25_64/Tempdata.total_pop
    Tempdata['Associates_College_perc']=(Tempdata.associates_degree +Tempdata.bachelors_degree)/Tempdata.total_pop
    Tempdata['Advanced_degree_perc']=(Tempdata.graduate_professional_degree+ Tempdata.masters_degree)/Tempdata.total_pop
    Tempdata['HS_perc']=Tempdata.high_school_including_ged/Tempdata.total_pop
    Tempdata['Perc_vacat_housing_units']=Tempdata.vacant_housing_units /Tempdata.housing_units
    Tempdata['Perc_other_race']=(Tempdata.amerindian_pop+Tempdata.other_race_pop)/Tempdata.total_pop
    Tempdata['Perc_black']=Tempdata.black_pop/Tempdata.total_pop
    Tempdata['Perc_asian']=Tempdata.asian_pop/Tempdata.total_pop
    Tempdata['Perc_hispanic']=Tempdata.hispanic_pop/Tempdata.total_pop
    Tempdata['Perc_commuter_by_public_transport']=Tempdata.commuters_by_public_transportation/Tempdata.commuters_16_over
    Tempdata['Perc_worked_at_home']=Tempdata.worked_at_home/Tempdata.civilian_labor_force
    Tempdata['Perc_different_house_year_ago_same_city']=Tempdata.different_house_year_ago_same_city/Tempdata.total_pop
    Tempdata['Perc_different_house_year_ago_dif_city']=Tempdata.different_house_year_ago_different_city/Tempdata.total_pop
    Tempdata['perc_housing_built_2005_or_later']=Tempdata.housing_built_2005_or_later/Tempdata.housing_units
    return Tempdata.drop(cols_to_drop,axis=1)


In [24]:
testdata=combo_and_drop_acs(testdata,cols_to_drop)
traindata=combo_and_drop_acs(traindata,cols_to_drop)
predictdata=combo_and_drop_acs(predictdata,cols_to_drop)


In [25]:
traindata.columns

Index(['RegionName', 'City', 'State', 'Rent', 'total_pop', 'median_age',
       'median_income', 'million_dollar_housing_units',
       'dwellings_50_or_more_units', 'mobile_homes',
       'housing_built_1939_or_earlier', 'armed_forces', 'gini_index',
       'poverty', 'walked_to_work', 'Percent_Female',
       'families_with_young_children_summed', 'dwellings_1_units',
       'dwellings_under_10_units', 'unemployment_rate',
       'average_car_per_household', 'Perc_commuters_under_15_min',
       'Perc_commuters_over_45_min', 'Percent_pop_25_64',
       'Associates_College_perc', 'Advanced_degree_perc', 'HS_perc',
       'Perc_vacat_housing_units', 'Perc_other_race', 'Perc_black',
       'Perc_asian', 'Perc_hispanic', 'Perc_commuter_by_public_transport',
       'Perc_worked_at_home', 'Perc_different_house_year_ago_same_city',
       'Perc_different_house_year_ago_dif_city',
       'perc_housing_built_2005_or_later'],
      dtype='object')

In [26]:
#should we percentize race cols
#population density col for zipcode

In [27]:
traindata.to_csv('col_droptrain.csv',index=False)
testdata.to_csv('col_droptest.csv',index=False)
predictdata.to_csv('col_droppredict.csv',index=False)

In [28]:
# Return multicollinearity chart
X = traindata.drop(['Rent', 'RegionName', 'City', 'State'], axis=1)

#used_list=traindata.columns
feature_names = X.columns
RSquared = []
variable_name = []
for var in feature_names:
    dep_var = StandardScaler().fit_transform(np.array(X[var]).reshape(-1,1))
    predictors =StandardScaler().fit_transform(X.drop(var, axis=1))
    ols = LinearRegression()
    ols.fit(predictors, dep_var)
    variable_name.append(var)
    RSquared.append(ols.score(predictors, dep_var))

val_dfs = pd.concat([pd.Series(RSquared), pd.Series(variable_name)], axis=1)
val_dfs = val_dfs.sort_values(by=0, ascending=False)
 
# ## Plotting multicollinearity test from previous project ##
# plt.plot(val_dfs[1], val_dfs[0])
# plt.xticks(rotation=90)
# plt.ylim(0.5,1)
# plt.rcParams["figure.figsize"] = (30,3)
# plt.show()

In [29]:
val_dfs.head(60)  

,0,1
0,0.966661,total_pop
16,0.921408,average_car_per_household
20,0.912008,Associates_College_perc
21,0.902785,Advanced_degree_perc
13,0.901946,dwellings_1_units
12,0.900105,families_with_young_children_summed
22,0.894765,HS_perc
28,0.894371,Perc_commuter_by_public_transport
2,0.880017,median_income
9,0.858728,poverty


In [ ]:
# ##drop notes
#  'nonfamily_households',
#  'family_households',
#  'median_year_structure_built',
#  'rent_burden_not_computed',-hella sparse,want to predict w no rent related cols also for below
#  'rent_over_50_percent', -hella sparse
#  'rent_40_to_50_percent',-hella sparse
#  'rent_35_to_40_percent',-hella sparse
#  'rent_30_to_35_percent',-hella sparse
#  'rent_25_to_30_percent',-hella sparse
#  'rent_20_to_25_percent',-hella sparse
#  'rent_15_to_20_percent',-hella sparse
#  'rent_10_to_15_percent',-hella sparse
#  'rent_under_10_percent',-hella sparse

#  'male_pop', -into percent female
#  'female_pop',-into percent female
#  'white_pop',-should add up to total pop
#  'amerindian_pop',- small 
#  'other_race_pop',-small and redundant, cant drop to offset sum(race) =total pop bc zero in some cases
#  'two_or_more_races_pop',
#  'not_hispanic_pop',-redundant w race cols
#  'commuters_by_public_transportation',
#  'households',-drop bc have family and non family households
#  'vacant_housing_units',-drop have total vacant units for sale and rent
#  'median_rent',-drop 
#  'percent_income_spent_on_rent',- drop cause we predicting this 
#  'owner_occupied_housing_units',- have for sale and rent ,

#  ['families_with_young_children',
#  'two_parent_families_with_young_children',
#  'two_parents_in_labor_force_families_with_young_children',
#  'two_parents_father_in_labor_force_families_with_young_children',
#  'two_parents_mother_in_labor_force_families_with_young_children',
#  'two_parents_not_in_labor_force_families_with_young_children',
#  'one_parent_families_with_young_children',
#  'father_one_parent_families_with_young_children',
#  'father_in_labor_force_one_parent_families_with_young_children',] can all become a fam w children col. the og one doesnt = the sum of all these 
#  'commute_10_14_mins',-drop for commute less than 10 or more than 60
#  'commute_15_19_mins',-drop for commute less than 10 or more than 60
#  'commute_20_24_mins',-drop for commute less than 10 or more than 60
#  'commute_25_29_mins',-drop for commute less than 10 or more than 60
#  'commute_30_34_mins',-drop for commute less than 10 or more than 60
#  'commute_45_59_mins',-drop for commute less than 10 or more than 60
#  'aggregate_travel_time_to_work',-highly population size dependant 
#  'renter_occupied_housing_units_paying_cash_median_gross_rent',-nope

#  'occupied_housing_units',-have for sale and for rent and totoal 

#  ['dwellings_1_units_detached',
#  'dwellings_1_units_attached',
#  'dwellings_2_units',
#  'dwellings_3_to_4_units',
#  'dwellings_5_to_9_units',
#  'dwellings_10_to_19_units',
#  'dwellings_20_to_49_units',
#  'dwellings_50_or_more_units',] can become three cats, one units mid units >50 units 

#  'housing_built_2000_to_2004',-drop- not new anymore so can blend w the other cols 
 
#  'male_under_5',-should sum to total male pop and also maybe issues w child homes so we droppin 
#  'female_under_5',^ same 
#  ['employed_pop',
#  'unemployed_pop',] become unemployment percentage 
#  'not_in_labor_force',- vif w age groups? drop 
#  'pop_16_over',-def vif w age groups
#  'pop_in_labor_force',-interaction w labor force
#  ['asian_male_45_54',
#  'asian_male_55_64',
#  'black_male_45_54',
#  'black_male_55_64',
#  'hispanic_male_45_54',
#  'hispanic_male_55_64',
#  'white_male_45_54',
#  'white_male_55_64',] interact w race and age ranges dropping 
#  'bachelors_degree_2',
#  'bachelors_degree_or_higher_25_64',
#  'children',-vif w age ranges 
#  ['commuters_by_bus',
#  'commuters_by_car_truck_van',
#  'commuters_by_carpool',
#  'commuters_by_subway_or_elevated',
#  'commuters_drove_alone',] - make into drive or public transport 
#  'four_more_cars',
#  'group_quarters',- what this 
#  'households_public_asst_or_food_stamps',
#  ['in_grades_1_to_4',
#  'in_grades_5_to_8',
#  'in_grades_9_to_12',] -will interact w age range
#  'less_than_high_school_graduate',- this with hs degree, bachelors, masters etc should be total pop so dropping this 
#  ['male_45_64_associates_degree',
#  'male_45_64_bachelors_degree',
#  'male_45_64_graduate_degree',
#  'male_45_64_less_than_9_grade',
#  'male_45_64_grade_9_12',
#  'male_45_64_high_school',
#  'male_45_64_some_college',
#  'male_45_to_64',] interact w age and school cols 
#  ['no_car',
#  'no_cars',] combine into avrg cars per person  
#  ['one_car',
#  'two_cars',
#  'three_cars',]combine into avrg cars per person 
#  'pop_25_64',-huge interactions 
#  'population_1_year_and_over',,-huge interactions 
#  'population_3_years_over',,-huge interactions 
#  'workers_16_and_over',-interactions
#  'high_school_diploma',-interactions w hs degree or ged 
#  'less_one_year_college',-interactions
#  'one_year_more_college',-interactions
#  'pop_25_years_over',-interactions
#  'commute_35_44_mins',-drop for commute less than 10 or more than 60
#  'management_business_sci_arts_employed-drop for vif 
#  'graduate_professional_degree-drop bc interacts w other school 
#  'commuters_16_over',-interactions
#  'hispanic_any_race'-interactions]